In [20]:
import pandas as pd

In [21]:
data = pd.read_csv('NvidiaDocumentationQandApairs.csv')

In [22]:
data

,Unnamed: 0,question,answer
0,0,What is Hybridizer?,Hybridizer is a compiler from Altimesh that en...
1,1,How does Hybridizer generate optimized code?,Hybridizer uses decorated symbols to express p...
2,2,What are some parallelization patterns mention...,The text mentions using parallelization patter...
3,3,How can you benefit from accelerators without ...,You can benefit from accelerators' compute hor...
4,4,What is an example of using Hybridizer?,An example in the text demonstrates using Para...
...,...,...,...
7103,7103,What is the focus of the GTC event in 2015?,The focus of the GTC event in 2015 is GPU code...
7104,7104,How were the main changes made to the code for...,"The main changes included merging kernels, reg..."
7105,7105,What are some key fields in the cudaDeviceProp...,"Some key fields include name, memoryClockRate,..."
7106,7106,What did changing the kernel approach achieve ...,Changing the kernel approach reduced the itera...


In [4]:
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertModel
from sentence_transformers import models, losses, SentenceTransformer
from sentence_transformers import SentenceTransformer, losses, SentencesDataset
from sentence_transformers.readers import InputExample

In [5]:
# Define your question-answer dataset
# Assuming you have lists of questions and answers
questions = data['question'].values.tolist()
answers = data['answer'].values.tolist()

In [6]:
questions_train = questions[:5000]
questions_test = questions[5001:]

In [7]:
len(questions_train)

5000

In [8]:
len(questions_test)

2107

In [9]:
answers_train = answers[:5000]
answers_test = answers[5001:]

In [10]:
len(answers_train)

5000

In [11]:
len(answers_test)

2107

In [12]:
model_name = "sentence-transformers/all-MiniLM-L12-v2"

# Load SBERT model
model = SentenceTransformer(model_name)

/Users/satishsilveri/anaconda3/lib/python3.11/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [13]:
def collate_fn(batch):
    texts = [example.texts for example in batch]
    labels = [example.label for example in batch]
    return {'texts': texts, 'labels': labels}

In [14]:
# Define a PyTorch DataLoader for your dataset
data = [InputExample(texts=[q, a], label=1) for q, a in zip(questions_train, answers_train)]

train_dataset = SentencesDataset(data, model=model)

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=8, collate_fn=collate_fn)

In [16]:
# Define loss function
train_loss = losses.MultipleNegativesRankingLoss(model=model)

model.fit(
    [(train_dataloader, train_loss)],
    epochs=10,
)

# Save the fine-tuned model
model.save('NvidiaQA_all-MiniLM-L12-v2')

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/625 [00:00<?, ?it/s]

Iteration:   0%|          | 0/625 [00:00<?, ?it/s]

Iteration:   0%|          | 0/625 [00:00<?, ?it/s]

Iteration:   0%|          | 0/625 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/625 [00:00<?, ?it/s]

Iteration:   0%|          | 0/625 [00:00<?, ?it/s]

Iteration:   0%|          | 0/625 [00:00<?, ?it/s]

Iteration:   0%|          | 0/625 [00:00<?, ?it/s]

Iteration:   0%|          | 0/625 [00:00<?, ?it/s]

In [34]:
import numpy as np
def create_negative_samples(row, df):
    # Get all answers except the current answer for this question
    df = df.sample(n=100)
    other_answers = df['answer'].tolist()
    if row['answer'] in other_answers:
        other_answers.remove(row['answer'])
    
    # Randomly select an answer from the list of other answers
    negative_answer = np.random.choice(other_answers)
    
    return negative_answer

In [35]:
data['negative_answer'] = data.apply(lambda row: create_negative_samples(row, data), axis=1)

In [36]:
data

,Unnamed: 0,question,answer,negative_answer
0,0,What is Hybridizer?,Hybridizer is a compiler from Altimesh that en...,The winning team used a TITAN X GPU.
1,1,How does Hybridizer generate optimized code?,Hybridizer uses decorated symbols to express p...,Jetson Xavier NX provides impressive computati...
2,2,What are some parallelization patterns mention...,The text mentions using parallelization patter...,While this guide can be followed from any Kit ...
3,3,How can you benefit from accelerators without ...,You can benefit from accelerators' compute hor...,"In device launch, the launch latency remains a..."
4,4,What is an example of using Hybridizer?,An example in the text demonstrates using Para...,CUDA-aware MPI implementations like MVAPICH2 o...
...,...,...,...,...
7103,7103,What is the focus of the GTC event in 2015?,The focus of the GTC event in 2015 is GPU code...,libnvidia-container is responsible for setting...
7104,7104,How were the main changes made to the code for...,"The main changes included merging kernels, reg...","GDB provides insight into thread behaviors, st..."
7105,7105,What are some key fields in the cudaDeviceProp...,"Some key fields include name, memoryClockRate,...","The text mentions the UCI Higgs dataset, a bin..."
7106,7106,What did changing the kernel approach achieve ...,Changing the kernel approach reduced the itera...,nvGRAPH supports essential graph algorithms li...


In [40]:
train_data = data.iloc[0:5000]

test_data = data.iloc[5001:]

In [43]:
train_data

,Unnamed: 0,question,answer,negative_answer
0,0,What is Hybridizer?,Hybridizer is a compiler from Altimesh that en...,The winning team used a TITAN X GPU.
1,1,How does Hybridizer generate optimized code?,Hybridizer uses decorated symbols to express p...,Jetson Xavier NX provides impressive computati...
2,2,What are some parallelization patterns mention...,The text mentions using parallelization patter...,While this guide can be followed from any Kit ...
3,3,How can you benefit from accelerators without ...,You can benefit from accelerators' compute hor...,"In device launch, the launch latency remains a..."
4,4,What is an example of using Hybridizer?,An example in the text demonstrates using Para...,CUDA-aware MPI implementations like MVAPICH2 o...
...,...,...,...,...
4995,4995,What is the role of the TCS feature-rich semi-...,The TCS feature-rich semi-automatic labeling t...,The two-step approach in spectral partitioning...
4996,4996,What was the motivation behind developing the ...,The motivation behind developing the automated...,Sparse matrix processing is optimized using pa...
4997,4997,What kind of datasets are crucial for camera-b...,Accurately annotated datasets are crucial for ...,"When adding PCAST to multi-threaded programs, ..."
4998,4998,What were the challenges with the initial vers...,The challenges with the initial version of the...,GPU-accelerated libraries in the CUDA Toolkit ...


In [45]:
model = SentenceTransformer(model_name)

In [50]:
constrastive_loss_train_data = []
for index, row in train_data.iterrows():
    temp_pos = InputExample(texts=[row['question'], row['answer']], label=1)
    temp_neg = InputExample(texts=[row['question'], row['negative_answer']], label=0)
    constrastive_loss_train_data.append(temp_pos)
    constrastive_loss_train_data.append(temp_neg)

In [51]:
constrastive_loss_train_dataset = SentencesDataset(constrastive_loss_train_data, model=model)

constrastive_loss_train_dataloader = DataLoader(constrastive_loss_train_dataset, shuffle=True, batch_size=8, collate_fn=collate_fn)

In [52]:
# Define loss function
constrastive_train_loss = losses.ContrastiveLoss(model=model)

model.fit(
    [(constrastive_loss_train_dataloader, constrastive_train_loss)],
    epochs=10,
)

# Save the fine-tuned model
model.save('NvidiaQA_all-MiniLM-L12-v2_constrastive_loss')

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1250 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1250 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1250 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1250 [00:00<?, ?it/s]

...

Iteration:   0%|          | 0/1250 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1250 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1250 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1250 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1250 [00:00<?, ?it/s]